<a href="https://colab.research.google.com/github/quanap5/ColabNote2/blob/master/Back_Propagation_su_dung_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

## Backpropagation với Tensorflow

[ref](http://blog.aloni.org/posts/backprop-with-tensorflow)

In [0]:
#!/usr/bin/env python

import tensorflow
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# create place to feed input and contain label
a_0 = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

# Parameters of network
hidden = 30
w_1 = tf.Variable(tf.truncated_normal([784, hidden]))
b_1 = tf.Variable(tf.truncated_normal([1, hidden]))
w_2 = tf.Variable(tf.truncated_normal([hidden, 10]))
b_2 = tf.Variable(tf.truncated_normal([1, 10]))

### 1.1 Định nghĩa hàm Sigmoid

In [0]:
def sigma(x):
    return tf.div(tf.constant(1.0),
                  tf.add(tf.constant(1.0), tf.exp(tf.negative(x))))

In [0]:
def quan_function(x):
  return x

### 1.2 Thực hiện tính toán Forward propagation

In [0]:
z_1 = tf.add(tf.matmul(a_0, w_1), b_1)
a_1 = sigma(z_1)
z_2 = tf.add(tf.matmul(a_1, w_2), b_2)
a_2 = sigma(z_2)

### 1.3 Tính gradient

In [0]:
diff = tf.subtract(a_2, y)

### 1.4 Tính derivative của hàm Sigmoid

In [0]:
def sigmoidprime(x):
  return tf.multiply(sigma(x), tf.subtract(tf.constant(1.0), sigma(x)))

### 1.5 Thực hiện BackPropagation

In [0]:
d_z_2 = tf.multiply(diff, sigmoidprime(z_2))
d_b_2 = d_z_2
d_w_2 = tf.matmul(tf.transpose(a_1), d_z_2)

d_a_1 = tf.matmul(d_z_2, tf.transpose(w_2))
d_z_1 = tf.multiply(d_a_1, sigmoidprime(z_1))
d_b_1 = d_z_1
d_w_1 = tf.matmul(tf.transpose(a_0), d_z_1)

### 1.6 Cập nhật network

In [0]:
eta = tf.constant(0.5)
step = [
    tf.assign(w_1,
            tf.subtract(w_1, tf.multiply(eta, d_w_1)))
  , tf.assign(b_1,
            tf.subtract(b_1, tf.multiply(eta,
                               tf.reduce_mean(d_b_1, axis=[0]))))
  , tf.assign(w_2,
            tf.subtract(w_2, tf.multiply(eta, d_w_2)))
  , tf.assign(b_2,
            tf.subtract(b_2, tf.multiply(eta,
                               tf.reduce_mean(d_b_2, axis=[0]))))
]

### 1.7  Chạy training và testing

In [14]:
# accuracy matrix 
acct_mat = tf.equal(tf.argmax(a_2, 1), tf.argmax(y, 1))
# sum up the correct prediction
acct_res = tf.reduce_sum(tf.cast(acct_mat, tf.float32))

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for i in range(10000):
  batch_xs, batch_ys = mnist.train.next_batch(10)
  sess.run(step, feed_dict = {a_0: batch_xs,
                              y : batch_ys})
  if i % 1000 == 0:
    res = sess.run(acct_res, feed_dict =
                   {a_0: mnist.test.images[:100],
                    y : mnist.test.labels[:100]})
    print(res)
    
print("FINISH")

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


10.0
73.0
86.0
91.0
95.0
94.0
93.0
95.0
97.0
97.0
FINISH
